<a href="https://colab.research.google.com/github/ivavasic/ivavasic/blob/main/train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#from __future__ import print_function

import numpy as np

from tensorflow import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K
from keras.callbacks import TensorBoard
from keras.models import model_from_json

from sklearn.metrics import confusion_matrix

In [ ]:
# input image dimensions
img_rows, img_cols = 28, 28

# the data, shuffled and split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

num_classes = 10

11501568/11490434 [==============================] - 0s 0us/step


In [ ]:
K.image_data_format()

'channels_last'

In [ ]:
if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

In [ ]:
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples


In [ ]:
# image  with a 9
# target = 9

#output of our CNN is [0.01 0.02.... 0.95] <-> target
#target -> one hot encoding [0 0 0 ... 1]

In [ ]:
from keras.utils import np_utils

# convert class vectors to categorical values
y_train = np_utils.to_categorical(y_train, num_classes)
y_test = np_utils.to_categorical(y_test, num_classes)

In [ ]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 26, 26, 32)        320       
                                                                 
 conv2d_1 (Conv2D)           (None, 24, 24, 64)        18496     
                                                                 
 max_pooling2d (MaxPooling2D  (None, 12, 12, 64)       0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 12, 12, 64)        0         
                                                                 
 flatten (Flatten)           (None, 9216)              0         
                                                                 
 dense (Dense)               (None, 128)               1179776   
                                                        

In [ ]:
batch_size = 128
epochs = 4#20

In [ ]:
LOG_DIR = 'log_dir'

In [ ]:
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

tbCallBack = TensorBoard(log_dir=LOG_DIR, 
                         histogram_freq=1,
                         write_graph=True,
                         write_grads=True,
                         batch_size=batch_size,
                         write_images=True)

model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test), #here I should have a validation set (x_valid, y_valid)! 
          callbacks=[tbCallBack])

score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])


Epoch 1/4
469/469 [==============================] - 17s 12ms/step - loss: 2.2843 - accuracy: 0.1689 - val_loss: 2.2535 - val_accuracy: 0.2772
Epoch 2/4
469/469 [==============================] - 5s 11ms/step - loss: 2.2332 - accuracy: 0.2637 - val_loss: 2.1883 - val_accuracy: 0.4387
Epoch 3/4
469/469 [==============================] - 5s 12ms/step - loss: 2.1653 - accuracy: 0.3597 - val_loss: 2.1008 - val_accuracy: 0.5437
Epoch 4/4
469/469 [==============================] - 6s 12ms/step - loss: 2.0744 - accuracy: 0.4318 - val_loss: 1.9833 - val_accuracy: 0.6246
Test loss: 1.9833201169967651
Test accuracy: 0.6245999932289124


In [ ]:
!mkdir "results"

In [ ]:
model_json = model.to_json()
with open("results/model.json", "w") as json_file:
    json_file.write(model_json)
        
model.save_weights('results/model_weights.h5')

model.save('results/model')

INFO:tensorflow:Assets written to: results/model/assets


In [ ]:
!zip -r results.zip results

  adding: results/ (stored 0%)
  adding: results/model_weights.h5 (deflated 9%)
  adding: results/model/ (stored 0%)
  adding: results/model/keras_metadata.pb (deflated 90%)
  adding: results/model/variables/ (stored 0%)
  adding: results/model/variables/variables.data-00000-of-00001 (deflated 7%)
  adding: results/model/variables/variables.index (deflated 65%)
  adding: results/model/assets/ (stored 0%)
  adding: results/model/saved_model.pb (deflated 88%)
  adding: results/model.json (deflated 81%)


In [ ]:
# load json and create model
json_file = open('results/model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("results/model_weights.h5")
print("Loaded model from disk")

Loaded model from disk


In [ ]:
y_prob = loaded_model.predict(x_test)
print(y_prob.shape)

y_pred = np.argmax(y_prob, axis = 1)
print(y_pred.shape)

(10000, 10)
(10000,)


In [ ]:
y_test_ = np.argmax(y_test, axis = 1)

In [ ]:
confusion_matrix(y_test_, y_pred)

array([[ 785,    2,    0,   16,   21,    0,  144,    0,   12,    0],
       [   0, 1112,    0,    4,    3,    0,    5,    0,   11,    0],
       [  23,  148,  413,  242,   72,    0,   63,    2,   69,    0],
       [   5,   56,    0,  886,    4,    0,    6,    1,   46,    6],
       [   0,   33,    0,    0,  903,    0,   24,    0,   14,    8],
       [  29,   55,    0,  264,  122,    0,   94,    0,  321,    7],
       [  10,   42,    1,    3,   86,    0,  808,    0,    8,    0],
       [  13,  177,    3,   87,  133,    0,    1,  431,   88,   95],
       [  14,   96,    0,   92,   57,    0,   37,    2,  668,    8],
       [   7,   97,    0,   25,  601,    0,    9,    0,   30,  240]])